# SVR prediction

In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd
import numpy as np
from numpy.random import shuffle

from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import ElasticNet
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, ARDRegression
from sklearn.base import clone
from sklearn.gaussian_process import GaussianProcessRegressor

from scipy.stats import ttest_ind

from datasets import subject_ET_FUS
from mytoolkit1 import correlation

subnetwork_path = r'E:\006_ET_MRgFUS\09_gradient_batch2\02_nii_mask\7network_atlas1039.csv'
subnetwork_df = pd.read_csv(subnetwork_path)
subnetwork_labels = subnetwork_df['Subnetwork'].values

subjects = subject_ET_FUS.load_subjects(r"E:\006_ET_MRgFUS\02_data_management\data_management_add_NC2.csv")

obs_name1 = 'ncnc'
obs_name2 = 'base'
obs_name3 = '180d'

filename1 = 'batch2.1/aligned_gradient1.csv'
filename2 = 'batch2.1/aligned_gradient2.csv'

def get_gradient(df1_path, df2_path):
    df1 = pd.read_csv(df1_path, index_col=0)
    df2 = pd.read_csv(df2_path, index_col=0)

    g1 = df1['gradient'].values.tolist()
    g2 = df2['gradient'].values.tolist()
    return np.array(g1+g2)

def get_xy(subjects, obs_name1, clinical_name='hand_tremor_R'):
    x = []
    y = []
    for subject in subjects:
        obs = subject.get_observation(obs_name1)
        gradient = obs.gradient
        gradient1_path = gradient.build_path(filename1)
        gradient2_path = gradient.build_path(filename2)

        x.append(get_gradient(gradient1_path, gradient2_path))
        #print(obs.args['hand_tremor_R'])
        y.append(obs.args[clinical_name])
    return x, y

def get_roi_gradient(df1_path, df2_path, roi):
    df1 = pd.read_csv(df1_path, index_col=0)
    df2 = pd.read_csv(df2_path, index_col=0)

    g1 = df1['gradient'].values[roi]
    g2 = df2['gradient'].values[roi]
    return g1, g2

def get_roi_xy(subjects, obs_name1, obs_name2, roi=980, clinical_name='hand_tremor_R'):
    x = []
    y = []
    for subject in subjects:
        obs = subject.get_observation(obs_name1)
        gradient = obs.gradient
        gradient1_path = gradient.build_path(filename1)
        gradient2_path = gradient.build_path(filename2)

        g1, g2 = get_roi_gradient(gradient1_path, gradient2_path, roi)

        gender_int = obs.args['gender_int']
        height = obs.args['height']
        weight = obs.args['weight']
        Duration = obs.args['Duration']
        Age = obs.args['Age']
        family_history = obs.args['family_history']
        SDR = obs.args['SDR']

        hand_tremor_R = obs.args['hand_tremor_R']
        hand_tremor_L = obs.args['hand_tremor_L']
        #hand_tremor_total = obs.args['hand_tremor_total']
        CRST_A = obs.args['CRST_A']
        CRST_B_R = obs.args['CRST_B_R']
        CRST_B_L = obs.args['CRST_B_L']
        CRST_B_total = obs.args['CRST_B_total']
        CRST_C = obs.args['CRST_C']
        CRST_TOTAL = obs.args['CRST_TOTAL']

        '''
        subject_x = [g1, g2, gender_int, height, weight, Duration, Age, family_history, SDR,
        hand_tremor_R, hand_tremor_L, hand_tremor_total, CRST_A, CRST_B_R, CRST_B_L, CRST_B_total,
        CRST_C, CRST_TOTAL]
        '''
        subject_x = [g1, g2, gender_int, Age]
        #[g1, g2, gender_int,  Age, Duration,family_history]

        obs2 = subject.get_observation(obs_name2)
        y1 = obs.args[clinical_name]
        y2 = obs2.args[clinical_name]
        subject_y = (y1-y2)/y1 * 100

        x.append(subject_x)
        y.append(subject_y)

    x = np.array(x)
    y = np.array(y)
    return x, y

def fit(x, y, fold=5, regression='SVR', kernel='linear', out_dir=None,log=True):
    
    if out_dir is not None:
        np.save(os.path.join(out_dir, 'x.npy'), x)
        np.save(os.path.join(out_dir, 'y.npy'), y)
    

    i = 0 
    x_shape = np.arange(np.shape(x)[1]) 
    
    r1s = []
    p1s = []
    maes = []
    r2s = []
    p2s = []
    best_models = []

    kf = KFold(n_splits=fold) 
    for train_index, test_index in kf.split(x): 
        
        x_train = x[train_index]
        y_train = y[train_index]

        x_test = x[test_index]
        y_test = y[test_index]

        best_score = -10000 
                           
        best_model = None
        best_en = None
        
        if out_dir is not None:
            np.save(os.path.join(out_dir, f'train_index_fold{i}.npy'), train_index)
            np.save(os.path.join(out_dir, f'test_index_fold{i}.npy'), test_index)
            
        kf_v = KFold(n_splits=8)
        for train_index2, valid_index in kf_v.split(x_train): 
            x_train2 = x_train[train_index2]
            y_train2 = y_train[train_index2]

            x_valid = x_train[valid_index] 
            y_valid = y_train[valid_index]

            en = ElasticNet(alpha=5, l1_ratio=0.9) 
                                                   
            en.fit(x_train2, y_train2) 
                                      
            selected_x_train2 = x_train2[:, en.coef_!=0] 

            selected_x_valid = x_valid[:, en.coef_!=0]

            if selected_x_train2.shape[1] == 0:
                continue
            
      
            if regression == 'SVR':
                model = SVR(kernel=kernel)

            elif regression == 'GPR':
                model = GaussianProcessRegressor(kernel=kernel)

            elif regression == 'Linear':
                model = LinearRegression()

            model.fit(selected_x_train2, y_train2)

            Rtwo = model.score(selected_x_valid, y_valid) 
            if Rtwo > best_score:
                best_score = Rtwo
                best_en = en       
                best_model = model 
                
            ''' 
            mae = mean_absolute_error(y_valid, y_pred)
            if mae < best_score:
                best_score = mae
                best_model = svr
            '''
        
        selected_x_test = x_test[:, best_en.coef_!=0]
        y_pred = best_model.predict(selected_x_test)
        r_square = best_model.score(selected_x_test, y_test)
        mae = mean_absolute_error(y_test, y_pred)
        
        if out_dir is not None:
            with open(os.path.join(out_dir, f'model_fold{i}.pkl'), 'wb') as ff:
                pickle.dump(best_model, ff)
            with open(os.path.join(out_dir, f'ElasticNet_fold{i}.pkl'), 'wb') as ff:
                pickle.dump(best_en, ff)

        
        if log:
            r1, p1 = correlation.correlation(y_test, y_pred, show=True, fig_width=6, fig_height=6, 
                            color="#1f77b4")
            print(x_shape[best_en.coef_!=0]) 
            print(f'R_square:{r_square}, MAE:{mae}') 
            print(r1, p1) 
        else:
            r1, p1 = correlation.correlation(y_test, y_pred, show=False, fig_width=8, fig_height=8, 
                            color="#1f77b4") 
        r1s.append(r1)
        p1s.append(p1)
        maes.append(mae)

        """
        selected_x_all = x[:, en.coef_!=0]
        y_pred_all = best_model.predict(selected_x_all)
        r2, p2 = correlation.correlation(y, y_pred_all, show=True)
        print(r2, p2)
        r2s.append(r2)
        p2s.append(p2)
        """
        i += 1 
        best_models.append(best_model) 

    if log:
        print("test_r1s")
        print(np.mean(r1s), np.std(r1s))
        #print(p1s)

        print("test_maes")
        print(np.mean(maes), np.std(maes))

        #print("test_r2s")
        #print(np.mean(r2s), np.std(r2s))
    return best_models, maes

def load_model(model_path):
    with open(model_path, 'rb') as ff:
        model = pickle.load(ff)
    return model
    '''
    model = load_model(r'E:\006_ET_MRgFUS\17_SVR_pickle\model_fold4.pkl')
    model.coef_
    '''
def permutation(in_dir, fold_i, times=2000):
        
    x = np.load(os.path.join(in_dir, 'x.npy'))
    y = np.load(os.path.join(in_dir, 'y.npy'))

    train_index = np.load(os.path.join(in_dir, f'train_index_fold{fold_i}.npy'))
    test_index = np.load(os.path.join(in_dir, f'test_index_fold{fold_i}.npy'))

    with open(os.path.join(in_dir, f'model_fold{fold_i}.pkl'), 'rb') as ff:
        best_model = pickle.load(ff)

    with open(os.path.join(in_dir, f'ElasticNet_fold{fold_i}.pkl'), 'rb') as ff:
        en = pickle.load(ff)

    x_train = x[train_index]
    y_train = y[train_index]
    x_test = x[test_index]
    y_test = y[test_index]

    selected_x_test = x_test[:, en.coef_!=0]
    y_pred = best_model.predict(selected_x_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = best_model.score(selected_x_test, y_test)

    selected_x_train = x_train[:, en.coef_!=0]
    maes = []
    r2s = []
    for i in range(times):
        shuffle(y_train)
        model = clone(best_model) 
       
        model.fit(selected_x_train, y_train)
        y_pred_ = model.predict(selected_x_test)
        mae_ = mean_absolute_error(y_test, y_pred_)
        r2_ = model.score(selected_x_test, y_test)
        maes.append(mae_)
        r2s.append(r2_)
    maes = np.array(maes)
    r2s = np.array(r2s)

    print("#########")
    temp_ = os.path.join(in_dir, "fold_i")
    print(temp_)
    
    r2_ratio = np.sum(r2s>r2) / times
    print("r2_ratio")
    print(r2_ratio)
    
    mae_ratio = np.sum(maes<mae) / times
    print("mae_ratio")
    print(mae_ratio)

    return mae_ratio, r2_ratio


In [ ]:
# 总体对应的情况
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

#K = [ "LU_R_posture"]

K = ["CRST_B_total","hand_tremor_R","CRST_TOTAL","CRST_C", "CRST_A"]

'''
"hand_tremor_L","CRST_B_R","CRST_B_L"
 "LU_R_tremor","LU_R_posture","LU_R_action","Write_R","Pour_R",
 "LU_L_tremor","LU_L_posture","LU_L_action","Write_L","Pour_L","head_tremor",
'''

for k in K:
    x1, y1 = get_xy(subjects, obs_name1='base', clinical_name=k)
    x2, y2 = get_xy(subjects, obs_name1='180d', clinical_name=k)
    x =x1+x2
    y =y1+y2

    x = np.array(x)
    y = np.array(y)
    print(k)
    _, maes = fit(x, y, 3, regression='SVR', out_dir=r"E:\006_ET_MRgFUS\17_SVR_pickle")
    mean_mae = np.mean(maes)

    times = 5000
    mean_per_maess = []
    for i in tqdm(range(times)):
        np.random.shuffle(y)
        _, per_maes = fit(x, y, 3, regression='SVR', log=False)
        mean_per_maes = np.mean(per_maes)
        mean_per_maess.append(mean_per_maes)
    mean_per_maess = np.array(mean_per_maess)
    mae_ratio = np.sum(mean_per_maess<mean_mae) / times
    
    print("####")
    print(mae_ratio) #
    print("======================================================")
            

In [ ]:
# 总体对应的情况
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

K = ["CRST_B_R","CRST_B_L"]

for k in K:
    x1, y1 = get_xy(subjects, obs_name1='base', clinical_name=k)
    x2, y2 = get_xy(subjects, obs_name1='180d', clinical_name=k)
    x =x1+x2
    y =y1+y2

    x = np.array(x)
    y = np.array(y)
    print(k)
    _, maes = fit(x, y, 3, regression='SVR', out_dir=r"E:\006_ET_MRgFUS\17_SVR_pickle")
    mean_mae = np.mean(maes)

    times = 5000
    mean_per_maess = []
    for i in tqdm(range(times)):
        np.random.shuffle(y)
        _, per_maes = fit(x, y, 3, regression='SVR', log=False)
        mean_per_maes = np.mean(per_maes)
        mean_per_maess.append(mean_per_maes)
    mean_per_maess = np.array(mean_per_maess)
    mae_ratio = np.sum(mean_per_maess<mean_mae) / times
    
    print("####")
    print(mae_ratio) #
    print("======================================================")
            

In [ ]:
# 术前预测术后
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

K = ["CRST_B_total","CRST_TOTAL", "hand_tremor_R",
"CRST_A","CRST_B_R","CRST_B_L","CRST_C",]

'''
"hand_tremor_L",
 "LU_R_tremor","LU_R_posture","LU_R_action","Write_R","Pour_R",
 "LU_L_tremor","LU_L_posture","LU_L_action","Write_L","Pour_L","head_tremor",
'''

for k in K:
    x1, y1 = get_xy(subjects, obs_name1='base', clinical_name=k)
    x2, y2 = get_xy(subjects, obs_name1='180d', clinical_name=k)
    x = np.array(x1)
    y1 = np.array(y1)
    y2 = np.array(y2)
    y = (y1-y2)/y1 * 100

    x = np.array(x)
    y = np.array(y)
    print(k)
    _, maes = fit(x, y, 3, regression='SVR', out_dir=r"E:\006_ET_MRgFUS\17_SVR_pickle")
    mean_mae = np.mean(maes)

    times = 5000
    mean_per_maess = []
    for i in tqdm(range(times)):
        np.random.shuffle(y)
        _, per_maes = fit(x, y, 3, regression='SVR', log=False)
        mean_per_maes = np.mean(per_maes)
        mean_per_maess.append(mean_per_maes)
    mean_per_maess = np.array(mean_per_maess)
    mae_ratio = np.sum(mean_per_maess<mean_mae) / times
    
    print("####")
    print(mae_ratio) #
    print("======================================================")
            
            

# Leave one out analysis

In [ ]:
import statsmodels.api as sm

kkk = ["hand_tremor_R", "hand_tremor_L", "hand_tremor_total", "CRST_A", "CRST_B_R", "CRST_B_L", "CRST_B_total",
        "CRST_C", "CRST_TOTAL"]

for k in kkk:
    print(k)
    for roi in range(1039):
        x, y = get_roi_xy(subjects, "base", "180d", roi=roi, clinical_name=k)
        x = sm.add_constant(x,prepend=False)

        ols = sm.OLS(y, x)
        
        result = ols.fit()
        #result.summary2()

        p0 = result.pvalues[0]
        p1 = result.pvalues[1]
        if p0<0.05 and p1<0.05:
            print(roi+1)

In [ ]:
import statsmodels.api as sm

rxxx=980
x, y = get_roi_xy(subjects, "base", "180d", roi=rxxx, clinical_name="hand_tremor_R")

x = sm.add_constant(x)

ols = sm.OLS(y, x)
result = ols.fit()
result.summary()

In [ ]:
K = [ "CRST_TOTAL"]

for cn in K:
    print(cn)
    x, y = get_roi_xy(subjects, "base", "180d", roi=980, clinical_name=cn)
    fit(x, y, fold=5, out_dir=r"E:\006_ET_MRgFUS\17_SVR_pickle", regression='Linear')

In [ ]:
#关注重复率+p值

import statsmodels.api as sm
from scipy.stats import pearsonr

kkk = ["hand_tremor_R", "CRST_TOTAL",  "CRST_A", "CRST_B_total", "CRST_C"]
# 
# "hand_tremor_L", "CRST_B_R", "CRST_B_L",

for k in kkk:
    print(k)
    for roi in range(1039):
        x, y = get_roi_xy(subjects, "base", "180d", roi=roi, clinical_name=k)
        x = sm.add_constant(x,prepend=False)

        count = 0
        y_tests = []
        y_preds = []
        for i in range(len(y)):
            x_test = x[i]
            y_test = y[i]

            new_x = np.delete(x, i, axis=0)
            new_y = np.delete(y, i, axis=0)

            ols = sm.OLS(new_y, new_x)

            result = ols.fit()
            y_pred = result.predict(x_test)

            p0 = result.pvalues[0]
            p1 = result.pvalues[1]
            if p0<0.05 or p1<0.05:
                count += 1
            y_tests.append(y_test)
            y_preds.append(y_pred)

        y_tests = np.array(y_tests).flatten()
        y_preds = np.array(y_preds).flatten()

        distances = np.abs(y_tests-y_preds)
        mae = np.mean(distances)
        std_ae = np.std(distances)

        r, p = pearsonr(y_tests, y_preds)

        if count > 22:
            print(roi+1, count, mae, std_ae, r, p)


In [ ]:
# 相关性的画图

import statsmodels.api as sm

rois = [421,994 ]


for roi in rois:
    
    x, y = get_roi_xy(subjects, "base", "180d", roi=roi, clinical_name="CRST_B_total")
    x = sm.add_constant(x,prepend=False)

    count = 0
    y_trues = []
    y_preds = []
    for i in range(len(y)):
        test_x = x[i]
        test_y = y[i]

        new_x = np.delete(x, i, axis=0)
        new_y = np.delete(y, i, axis=0)

        ols = sm.OLS(new_y, new_x)

        result = ols.fit()

        y_pred = result.predict(test_x)
        y_trues.append(test_y)
        y_preds.append(y_pred[0])

        p0 = result.pvalues[0]
        p1 = result.pvalues[1]
        if p0<0.05 and p1<0.05:
            count += 1
    if count > 2:
        print(roi+1, count)

    correlation.correlation(y_trues, y_preds, show=True, fig_width=6, fig_height=6,x_lim=[15,90],y_lim=[15,90]) #,color="#1f77b4"